
Open the notebook in Colab

<a href="https://colab.research.google.com/github/gbarbalinardo/kaldo/blob/feature/examples_tutorials_up_to_date/docs/amorphous_presentation_update_v2_summer_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remote fetch source code from Github

In [ ]:
!git clone https://github.com/nanotheorygroup/kaldo.git

# Install Kaldo from source

In [ ]:
!rm -r /content/sample_data

# Update utilities for tensorboard

!pip install --upgrade grpcio
!pip install --upgrade google-auth

# Install kaldo from source

%cd /content/kaldo
!pip install .

# Navigate back to workspace after installation completes
%cd /content/

# Fetch supplyment data remotely

In [ ]:
import os

# Fetch forcefield files

if not os.path.exists('forcefields.zip'):
  !wget http://sophe.ucdavis.edu/structures/forcefields.zip
  !unzip forcefields.zip
  !rm -r forcefields.zip

# Fetch force constants file for amprohous silicon (512 atoms)

if not os.path.exists('structure_a_si_512.zip'):
  !wget http://sophe.ucdavis.edu/structures/structure_a_si_512.zip
  !unzip structure_a_si_512.zip
  !rm -r structure_a_si_512.zip

# Remote fetch precomplied LAMMPS

In [ ]:
!wget http://sophe.ucdavis.edu/structures/lmp-tesla-t4-intel-xeon.gz
!tar xvzf lmp-tesla-t4-intel-xeon.gz
!rm /content/lmp-tesla-t4-intel-xeon.gz
!ln -s /content/lammps/src/lmp_gpu /usr/local/bin/lmp_gpu

# Naviagate to lammps source folder pior to LAMMPS-Python integration

%cd /content/lammps/src

print('\n')
print('Precomplied LAMMPS is fetched from remote!')

#Integrate LAMMPS Into Python

In [ ]:
!make install-python

# Copy executable file to where the python Module locates

import shutil 
src_path = '/usr/lib/python3/dist-packages/liblammps.so'
dist_path = '/usr/local/lib/python3.6/dist-packages/liblammps.so'
shutil.copyfile(src_path,dist_path)

# Naviagate back to main folder before simulation

%cd /content

print('\n')
print('LAMMPS-Python Integration is completed!')

# Thermal transport simulation for Amorphous-Silicon (a-Si)

## Read in force constant objects from folder

In [ ]:
from kaldo.forceconstants import ForceConstants

# Read amporphous silicon with eskm format

forceconstants = ForceConstants.from_folder(folder='structure_a_si_512', format='eskm')

## Create phonons object


In [ ]:
from kaldo.phonons import Phonons

# Create a phonon object
# is_classic flag can be true or false, as
# well as 0 (quantum) or 1 (classic)

phonons = Phonons (forceconstants=forceconstants,
                   is_classic=0,
                   temperature=300,
                   folder='si-amorphous',
                   third_bandwidth=0.05/4.135,
                   broadening_shape='triangle',
                   is_tf_backend=True,
                   storage='numpy')

## Acess and visualize properties calculated during simulations

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

# Direct access to properties
# calculated during the simulation

frequency = phonons.frequency.flatten(order='C')
bandwidth = phonons.bandwidth.flatten(order='C')

# Plot phonon bandwidth vs frequency

print('\n')
plt.plot(frequency[3:],bandwidth[3:],'.',markersize=10,label= 'broadening shape: ' + str(phonons.broadening_shape))
plt.ylabel('$\Gamma$ (THz)', fontsize=25, fontweight='bold')
plt.xlabel("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylim([bandwidth.min(), bandwidth.max()])
plt.legend(loc=2,frameon = False)
plt.show()

phase_space = phonons.phase_space.flatten(order='C')

# Plot phase space vs frequency

print('\n')
plt.figure()
plt.plot(frequency[3:], phase_space[3:], '.', markersize=10)
plt.ylabel ("Phase space", fontsize=25, fontweight='bold')
plt.xlabel("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.show()

print('\n')
# Plot diffusivity vs frequency

diffusiviy = phonons.diffusivity.flatten(order='C')
plt.figure()
plt.plot(frequency[3:], diffusiviy[3:], '.', markersize=10)
plt.ylabel ("$D(mm^{2}/s)$", fontsize=25, fontweight='bold')
plt.xlabel("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.show()

## Calculate and visualize $\kappa_{per \ mode}$ and $\kappa_{cum}$

In [ ]:
def cumulative_cond_cal(freq,full_cond,n_phonons):

  conductivity = np.einsum('maa->m', 1/3 * full_cond)
  conductivity = conductivity.reshape(n_phonons)
  cumulative_cond = np.zeros_like(conductivity)
  freq_reshaped = freq.reshape(n_phonons)

  for mu in range(cumulative_cond.size):
      single_cumulative_cond = conductivity[(freq_reshaped < freq_reshaped[mu])].sum()
      cumulative_cond[mu] = single_cumulative_cond
  
  return cumulative_cond

In [ ]:
from kaldo.conductivity import Conductivity
import numpy as np

# Compute conductivity with per phonon mode basis using qhgk method

kappa_qhgk_per_mode = np.einsum('maa->m',1/3*Conductivity(phonons=phonons, method='qhgk',n_iterations=20).conductivity)

# Compute cumulative conductivity by frequency with qhgk method

kappa_qhgk_cum_freq = cumulative_cond_cal(phonons.frequency,Conductivity(phonons=phonons, method='qhgk').conductivity,phonons.n_phonons)

print('\n')

# Visualize the per mode conductivity vs frequency

plt.figure()
plt.plot(frequency,kappa_qhgk_per_mode,'.',label='QHGK',ms=8)
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylabel(r'$\kappa(W/m/K)$', fontsize=25, fontweight='bold')
plt.legend(loc=1,frameon=False)
plt.show()

print('\n')

# Visualize the cumulative conductivity vs frequency

plt.figure()
plt.plot(frequency,kappa_qhgk_cum_freq,'.',label='QHGK',ms=8)
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylabel(r'$\kappa_{cum}(W/m/K)$', fontsize=25, fontweight='bold')
plt.legend(loc=4,frameon=False)
plt.grid()
plt.show()

## Run a-Si simulation with varied temperature and third bandwidth

In [ ]:
# Create a finite difference object
forceconstants = ForceConstants.from_folder(folder='structure_a_si_512', format='eskm')

# Create a phonon object
temperatures = [5, 10, 15, 20, 35, 50, 75, 100, 150, 200, 300, 500, 1000]
for third_bandwidth in [0.5, 1, 2, 4]:
    conds = []
    for temperature in temperatures:
        phonons = Phonons (forceconstants=forceconstants,
                   is_classic=0,
                   temperature=temperature,
                   folder='si-amorphous',
                   third_bandwidth=third_bandwidth/4.135,
                   broadening_shape='triangle')


        print('QHGK conductivity')
        cond = (phonons.conductivity(method='qhgk', third_bandwidth=third_bandwidth/4.135).sum(axis=0))
        conds.append(cond.diagonal().mean())
    plt.plot(temperatures, conds, label='$\delta_{in}$='+str(np.round(third_bandwidth/4.135,2))+' THz')
    
plt.ylabel('$\kappa$ (W/m/K)', fontsize=25, fontweight='bold')
plt.xlabel("$Temperature$ (K)", fontsize=25, fontweight='bold')
plt.legend(prop={'size': 20},frameon=False)
plt.show()